In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [2]:
STORM_NUMBER = 4
TITLE = "10 November 2020"
SUBTITLE = "8 AM 11 November 2020"
sts = utc(2020, 11, 10, 18)
ets = utc(2020, 11, 26, 23, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 25, 'z': 'yes', 'f': 'linear'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s, "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=0)
    return mp

In [8]:
cull = []
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["state"] == "MN"].index, USEME] = False
hardcode = [
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=2, upper=6.)
res = mp.postprocess(filename='201111.png')
mp.close()

    state  wfo  val      lon      lat                             geo  \
92     NE  FSD  5.6 -96.7500  42.5900  POINT (266027.797 -262679.490)   
97     SD  FSD  5.5 -96.7300  43.5900  POINT (263289.773 -151499.387)   
98     SD  FSD  5.5 -96.6900  43.4400  POINT (267176.206 -168037.237)   
103    SD  FSD  5.4 -96.9300  43.9200  POINT (245828.848 -115452.554)   
104    SD  FSD  5.3 -96.9200  42.7600  POINT (251424.451 -244327.742)   
106    SD  FSD  5.0 -96.8100  43.5100  POINT (257196.078 -160644.554)   
108    IA  FSD  5.0 -96.5600  42.8300  POINT (280470.481 -235379.491)   
119    SD  FSD  5.0 -96.9600  44.0000  POINT (243098.863 -106652.045)   
121    SD  FSD  5.0 -96.7800  43.6900  POINT (258832.639 -140547.276)   
122    SD  FSD  5.0 -96.7600  43.6900  POINT (260439.241 -140483.545)   
132    SD  FSD  4.6 -96.9400  42.7900  POINT (249670.505 -241055.457)   
135    IA  FSD  4.5 -95.8400  43.1800  POINT (337170.812 -193775.968)   
136    SD  FSD  4.5 -96.7000  43.5000  POINT (26610